In [2]:
import pandas as pd

In [24]:
mysheet = 'PID_Schedule_Job_20221003.xls'

Sheet 1_PID_Date to get NPI name

In [53]:
PID_Date = pd.read_excel(mysheet, sheet_name='1_PID_Date', header=1)
name = PID_Date.loc[:,['Project Name','NPI SMT']]
name = name.drop_duplicates().replace('.', None).dropna(axis=0)
name.head()

,Project Name,NPI SMT
0,Alpha Dog Collar,"Cheng, Danny(PE)"
6,"Approach R10, Asia","Lan, Herry"
12,Approach R50,"Chiang, Steve; Shen, Stephen;"
18,"Approach R50, Asia","Chiang, Steve; Shen, Stephen;"
24,Approach S70,"Chen, Andy(PE)"


In [52]:
df_SMT = name['NPI SMT'].str.split(';',expand=True).replace(' ', None).rename({0:'1',1:'2',2:'3',3:'4',4:'5'},axis=1)
df_SMT.head()

,1,2,3,4
0,"Cheng, Danny(PE)",None,None,None
6,"Lan, Herry",None,None,None
12,"Chiang, Steve","Shen, Stephen",None,None
18,"Chiang, Steve","Shen, Stephen",None,None
24,"Chen, Andy(PE)",None,None,None


In [71]:
merged_name = pd.merge(left=name, right=df_SMT, left_on=name.index, right_on=df_SMT.index, how='left').drop(['key_0','NPI SMT'],axis=1)
merged_name.head()

,Project Name,1,2,3,4
0,Alpha Dog Collar,"Cheng, Danny(PE)",None,None,None
1,"Approach R10, Asia","Lan, Herry",None,None,None
2,Approach R50,"Chiang, Steve","Shen, Stephen",None,None
3,"Approach R50, Asia","Chiang, Steve","Shen, Stephen",None,None
4,Approach S70,"Chen, Andy(PE)",None,None,None


In [72]:
melted_name = merged_name.melt(id_vars='Project Name', value_vars=['1','2','3','4'],var_name='No',value_name='SMTPE').dropna(axis=0).sort_values('Project Name')
melted_name['Project Name'] = melted_name['Project Name'].apply(lambda project : project.replace(' ',''))
melted_name

,Project Name,No,SMTPE
91,AirmarCM265LH,1,"Liu, Kent"
92,Alpha300/300i(i.MX7Platform),1,"Cheng, Danny(PE)"
198,Alpha300/300i(i.MX7Platform),2,"Mao, Daniel"
0,AlphaDogCollar,1,"Cheng, Danny(PE)"
1,"ApproachR10,Asia",1,"Lan, Herry"
...,...,...,...
86,fenix7XPro,1,"Huang, Shawn"
87,vivoactive5,1,"Lin, Anton"
88,vivomoveTrend,1,"Chen, Andy(PE)"
89,"vivomoveTrend,Asia(previouslyvivomove4)",1,"Chen, Andy(PE)"


Sheet 2_FRM178_Jobs to get project schedule

In [97]:
FRM_178 = pd.read_excel(mysheet, sheet_name='2_FRM178_Jobs', header=2)
FRM_178_SMT = FRM_178[FRM_178.Resource == 'SMT']
FRM_178_SMT = FRM_178_SMT.drop(['Item Number','Resource','Description_','Line Type','Org','(Days)','LCM PUR','LCM I-RING','Comment(Ref)','On Hand','Category','Job_Comment','ModelName','planner','Item Class','Total Quantity','MP Date','MP type'],axis=1)
FRM_178_SMT = FRM_178_SMT.iloc[:,:-1] #Remove 'after11/30'
FRM_178_SMT.drop_duplicates(subset='Model_NM_', keep='first', inplace=True)
FRM_178_SMT.head()

,Model_NM_,2022-09-30 00:00:00,2022-10-01 00:00:00,2022-10-02 00:00:00,2022-10-03 00:00:00,2022-10-04 00:00:00,2022-10-05 00:00:00,2022-10-06 00:00:00,2022-10-07 00:00:00,2022-10-08 00:00:00,...,2022-11-21 00:00:00,2022-11-22 00:00:00,2022-11-23 00:00:00,2022-11-24 00:00:00,2022-11-25 00:00:00,2022-11-26 00:00:00,2022-11-27 00:00:00,2022-11-28 00:00:00,2022-11-29 00:00:00,2022-11-30 00:00:00
28,HondaNaviProductionSupport,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,Other,NaN,NaN,NaN,400,v,v,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,Alpha2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,RA70N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,PanoptixPS22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
pd.merge(left=melted_name, right=FRM_178_SMT, left_on=melted_name['Project Name'], right_on=FRM_178_SMT['Model_NM_'], how='left').dropna(subset=['Model_NM_'])

,key_0,Project Name,No,SMTPE,Model_NM_,2022-09-30 00:00:00,2022-10-01 00:00:00,2022-10-02 00:00:00,2022-10-03 00:00:00,2022-10-04 00:00:00,...,2022-11-21 00:00:00,2022-11-22 00:00:00,2022-11-23 00:00:00,2022-11-24 00:00:00,2022-11-25 00:00:00,2022-11-26 00:00:00,2022-11-27 00:00:00,2022-11-28 00:00:00,2022-11-29 00:00:00,2022-11-30 00:00:00
9,ApproachS70,ApproachS70,1,"Chen, Andy(PE)",ApproachS70,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,ApproachS70S,ApproachS70S,1,"Cheng, Danny(PE)",ApproachS70S,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,ApproachZ50,ApproachZ50,1,"Pu, Alex",ApproachZ50,NaN,NaN,NaN,22,v,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,BMWMGU22,BMWMGU22,1,"Lin, David(PE)",BMWMGU22,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,Bounce,Bounce,1,"Huang, Shawn",Bounce,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,CatalystKart/Moto/Auto,CatalystKart/Moto/Auto,1,"Chiang, Steve",CatalystKart/Moto/Auto,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,CatalystKart/Moto/Auto,CatalystKart/Moto/Auto,3,"Shen, Stephen",CatalystKart/Moto/Auto,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,CatalystKart/Moto/Auto,CatalystKart/Moto/Auto,2,"Lee, Frankie",CatalystKart/Moto/Auto,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,ECHOMAPUHD2(Touch),ECHOMAPUHD2(Touch),1,"Chiang, Steve",ECHOMAPUHD2(Touch),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,ECHOMAPUHD2(Touch),ECHOMAPUHD2(Touch),2,"Pu, Alex",ECHOMAPUHD2(Touch),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
